<a href="https://colab.research.google.com/github/ShivaShankaraBr/faq_rakendra_beta01/blob/main/projeto_rakendra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%pip -q install google-genai

In [35]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [36]:
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"  # Usando gemini-2.0-flash em todos os agentes

In [37]:
!pip install -q google-adk
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
import warnings
warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [38]:
##########################################
# --- Agente 1: Buscador de FAQs --- #
##########################################
def agente_buscador(pergunta): # Renomeado 'topico' para 'pergunta' para maior clareza

    buscador = Agent(
        name="agente_buscador",
        model=MODEL_ID,  # Usando gemini-2.0-flash
        instruction="""
        Você é um assistente de pesquisa especializado em encontrar respostas para perguntas frequentes sobre xamanismo e plantas de poder.
        A sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar informações relevantes e confiáveis que respondam à pergunta do usuário.
        Priorize fontes como:
        -   Sites de instituições de pesquisa (e.g., universidades, centros de estudos)
        -   Artigos científicos introdutórios ou revisões (se acessíveis)
        -   Sites de organizações reconhecidas na área de saúde ou estudos religiosos
        -   Sites de especialistas renomados no campo do xamanismo (com credenciais verificáveis)
        Evite fontes como:
        -   Blogs pessoais sem credenciais claras
        -   Fóruns de discussão
        -   Sites de venda de produtos
        -   Fontes sensacionalistas ou com informações não verificadas
        Se a pergunta envolver plantas de poder, busque informações sobre seus usos tradicionais, composição química básica,
        efeitos conhecidos e potenciais riscos, sempre com foco em fontes confiáveis.
        Limite a busca a no máximo 5 resultados relevantes.
        """,
        description="Agente que busca informações no Google para responder a perguntas frequentes.",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Pergunta: {pergunta}" # Modificado para usar 'pergunta'
    resultados_da_busca = call_agent(buscador, entrada_do_agente_buscador)
    return resultados_da_busca

In [39]:
################################################
# --- Agente 2: Planejador de Posts de FAQ --- #
################################################
def agente_planejador(pergunta, resultados_da_busca): # Adicionado resultados da busca como entrada
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,  # Usando gemini-2.0-flash
        instruction="""
        Você é um planejador de conteúdo para redes sociais, especializado em criar posts informativos e envolventes a partir de informações encontradas na web.
        A sua tarefa é analisar os resultados da busca fornecidos para responder à pergunta do usuário e criar um plano para um post de Instagram.
        O plano deve conter os seguintes elementos:
        -   Pergunta: A pergunta original do usuário, para ser incluída no início do post.
        -   Resposta Concisa: Um resumo claro e direto da resposta à pergunta, baseado nas informações mais relevantes e confiáveis encontradas na busca.
        -   Pontos Principais: Uma lista dos 3 a 5 pontos mais importantes da resposta, que devem ser destacados no post.
        -   Linguagem: Sugestões de linguagem acessível e envolvente para o público do Instagram, evitando jargões técnicos.
        -   Recursos Visuais Sugeridos: Ideias para elementos visuais que poderiam acompanhar o post (lembre-se que não podemos gerar imagens, são apenas sugestões).
        -   Hashtags: Sugestões de 3 a 5 hashtags relevantes para o tema do post.
        """,
        description="Agente que planeja posts de Instagram informativos a partir de resultados de busca.",
        tools=[]  # Este agente não usa ferramentas
    )

    entrada_do_agente_planejador = f"Pergunta: {pergunta}\nResultados da busca: {resultados_da_busca}" # Passa a pergunta e os resultados
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [40]:
######################################
# --- Agente 3: Redator do Post de FAQ --- #
######################################
def agente_redator(pergunta, plano_de_post): # Adicionado pergunta como entrada
    redator = Agent(
        name="agente_redator",
        model=MODEL_ID,  # Usando gemini-2.0-flash
        instruction="""
        Você é um redator de conteúdo para redes sociais, especializado em criar posts informativos e envolventes para o Instagram.
        A sua tarefa é escrever um post para o Instagram que responda à pergunta do usuário, utilizando o plano de post fornecido.
        O post deve seguir as seguintes diretrizes:
        -   Começar com a pergunta do usuário de forma clara e direta.
        -   Apresentar a resposta de forma concisa e acessível, usando linguagem adequada para o público do Instagram (jovens adultos).
        -   Destacar os pontos principais da resposta, utilizando recursos como parágrafos curtos, listas ou emojis (se apropriado).
        -   Incluir um breve contexto sobre a Rakendra e sua perspectiva sobre o tema, se relevante e apropriado.
        -   Finalizar o post com um convite para seguir a página da Rakendra para mais informações e para participar de eventos (se aplicável).
        -   Incluir as hashtags sugeridas no plano de post.
        """,
        description="Agente redator de posts de Instagram informativos e envolventes."
    )
    entrada_do_agente_redator = f"Pergunta: {pergunta}\nPlano de post: {plano_de_post}" # Passa a pergunta para o redator
    rascunho_do_post = call_agent(redator, entrada_do_agente_redator)
    return rascunho_do_post

In [41]:
##########################################
# --- Agente 4: Revisor de Qualidade de FAQ --- #
##########################################
def agente_revisor(pergunta, rascunho_do_post): # Adicionado pergunta
    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,  # Usando gemini-2.0-flash
        instruction="""
        Você é um editor e revisor de conteúdo para redes sociais, com foco em posts informativos para o Instagram.
        A sua tarefa é revisar o rascunho do post fornecido, verificando os seguintes aspectos:
        -   Clareza: A resposta à pergunta é clara e fácil de entender para o público do Instagram?
        -   Precisão: A resposta está correta e alinhada com as informações encontradas na busca na web?
        -   Tom: O tom do post é adequado para o público do Instagram (jovens adultos) e para o tema (xamanismo, plantas de poder)?
        -   Linguagem: A linguagem é acessível, envolvente e livre de erros gramaticais e ortográficos?
        -   Formatação: O post está bem formatado, com parágrafos curtos, listas ou emojis (se apropriado)?
        -   Alinhamento com a Rakendra: O post apresenta a perspectiva da Rakendra sobre o tema de forma adequada e respeitosa?
        -   Inclusão de Hashtags: As hashtags sugeridas estão incluídas e são relevantes?
        Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
        Caso haja problemas, aponte-os e sugira melhorias específicas.
        """,
        description="Agente revisor de posts de Instagram informativos."
    )
    entrada_do_agente_revisor = f"Pergunta: {pergunta}\nRascunho do post: {rascunho_do_post}" # Passa a pergunta
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [42]:
def main():
    data_de_hoje = date.today().strftime("%d/%m/%Y")
    print("🚀 Iniciando o Sistema de Criação de Posts de FAQ para Instagram com 4 Agentes 🚀")

    # --- Obter a Pergunta do Usuário ---
    pergunta = input("❓ Por favor, digite a PERGUNTA sobre xamanismo ou plantas de poder que você deseja responder no post: ")

    # --- Lógica do sistema de agentes ---
    if not pergunta:
        print("Você esqueceu de digitar a pergunta!")
    else:
        print(f"Maravilha! Vamos criar um post para responder à pergunta: {pergunta}")

        resultados_da_busca = agente_buscador(pergunta)
        print("\n--- 🔍 Resultado do Agente 1 (Buscador) ---\n")
        display(to_markdown(resultados_da_busca))
        print("--------------------------------------------------------------")

        plano_do_post = agente_planejador(pergunta, resultados_da_busca)
        print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
        display(to_markdown(plano_do_post))
        print("--------------------------------------------------------------")

        rascunho_do_post = agente_redator(pergunta, plano_do_post)
        print("\n--- ✍️ Resultado do Agente 3 (Redator) ---\n")
        display(to_markdown(rascunho_do_post))
        print("--------------------------------------------------------------")

        post_final = agente_revisor(pergunta, rascunho_do_post)
        print("\n--- ✅ Resultado do Agente 4 (Revisor) ---\n")
        display(to_markdown(post_final))
        print("--------------------------------------------------------------")


In [ ]:
if __name__ == "__main__":
    main()


🚀 Iniciando o Sistema de Criação de Posts de FAQ para Instagram com 4 Agentes 🚀
